# Assignment Part 1 Brief 

## Deadline: Friday, December 3, 2021 at 15:00 hrs

## Number of marks available for Part 1: 25

## Scope: Sessions 1 to 5

### Please READ the whole assignment first, before starting to work on it.

### How and what to submit

A. A **Jupyter Notebook** with the code in all the cells executed and outputs displayed.

B. Name your Notebook **COM4509-6509_Assignment_Part1_Username_XXXXXX.ipynb** where XXXXXX is your username such as such as abc18de.  

C. Upload a .zip file to Blackboard before the deadline that contains the Jupyter Notebook in B and any other files requested for the solution of Part 2 of the Assignment (Dr Lu will be in charge of releasing this part). 

D. **NO DATA UPLOAD**: Please do not upload the data files used in this Notebook. We have a copy already. 


### Assessment Criteria 

* Being able to use numpy and pandas to preprocess a dataset.

* Being able to use numpy to build a machine learning pipeline for supervised learning. 

* Being able to follow the steps involved in an end-to-end project in machine learning.

* Being able to use scikit-learn to design a machine learning model pipeline


### Late submissions

We follow Department's guidelines about late submissions, i.e., a deduction of 5% of the mark each working day the work is late after the deadline. NO late submission will be marked one week after the deadline because we will release a solution by then. Please read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/late-submission) if you are taking COM4509 or read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/assessment/late-submission) if you are taking COM6509. 

### Use of unfair means 

**"Any form of unfair means is treated as a serious academic offence and action may be taken under the Discipline Regulations."** (from the students Handbook). Please carefully read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/unfair-means) on what constitutes Unfair Means if not sure, for COM4509. If you are taking COM6509, please read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/referencing-unfair-means) if you are not sure what is Unfair means. If you still have questions, please ask your Personal tutor or the Lecturers. 

# A dataset of air quality

The dataset you will use in this assignment comes from a popular machine learning repository that hosts open source datasets for educational and research purposes, the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). We are going to use regularised ridge regression and random forests for predicting air quality. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

In [35]:
import urllib.request
doq = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
urllib.request.urlretrieve(doq, pat_sav)
#urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip")

('./AirQualityUCI.zip', <http.client.HTTPMessage at 0x7f8d41dafca0>)

In [36]:
import zipfile
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')

In [37]:
# The .csv version of the file has some typing issues, so we use the excel version
import pandas as pd 
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15))

We can see some of the rows in the dataset 

In [38]:
air_quality_full.sample(5)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
7358,3.2,1473.75,-200,18.982866,1267.75,582.8,501.75,136.1,1676.75,1503.25,12.075,67.575001,0.949953
6365,1.9,1005.50,-200,6.534919,836.25,245.0,1226.00,113.0,1209.75,901.75,9.625,79.925001,0.957896
5271,-200.0,1062.50,-200,9.645572,964.25,-200.0,729.25,-200.0,1454.00,1045.00,17.425,62.250002,1.228224
2218,0.4,789.25,-200,2.551773,623.00,36.0,1145.50,53.0,1406.50,662.50,22.000,44.250000,1.155345
4692,1.5,1046.50,-200,9.658723,964.75,268.0,736.50,79.0,1605.50,1075.25,18.325,70.975002,1.480740


The target variable corresponds to the CO(GT) variable of the first column. The following columns correspond to the variables in the feature vectors, *e.g.*, PT08.S1(CO) is $x_1$ up until AH which is $x_D$. The original dataset also has a date and a time columns that we are not going to use in this assignment.

### Removing instances 

The dataset has missing values tagged with a -200 value. To simplify the design of the machine learning models in this assignment, we perform the following two operations to the dataset right from the beginning:

* we will remove the rows for which the target variable has missing values. We are doing supervised learning so we need all our data observations to have known target values.

* we will remove features with more than 20% of missing values. 

The code below performs both operations

In [39]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]

In [40]:
# We now remove the columns (features) for which there are more that 20% of missing values
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

### Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. *It is important to remember that the test data has to be set aside before preprocessing*. 

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage.  Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Furthermore, we are going to use *hold-out validation* for validating our predictive model so we need to further separate the training data into a training set and a validation set.

We split the dataset into a training set, a validation set and a test set. The training set will have 70% of the total observations, the validation set 15% and the test set, the remaining 15%. For making the random selections of the training and validation sets **make sure that you use a random seed that corresponds to the last five digits of your student UCard**. In the code below, I have used 55555 as an example of my random seed.

In [41]:
np.random.seed(88246)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

The assigment is divided into two sections. In the **first section**, you will design a regularised ridge regression model trained with stochastic gradient descent. You will write all the code from scratch. You should not use any library that already implements any of the routines considered in this section, for example, scikit-learn. In the **second section**, you will design a random forests model and you are allowed to use scikit-learn in this section.

When writing your code, you will find out that there are operations that are repeated at least twice. We will assign marks for use of Python functions and for commenting your code. The marks will be assigned as:

* Did you include Python functions to solve the question and avoid repeating code? (**1 mark**)
* Did you comment your code to make it readable to others? (**1 mark**)

# 1. Using regularised ridge regression to predict air quality (10 marks)

**DO NOT USE scikit-learn or any other machine learning library for the questions on this section. You are meant to write Python code from scratch. You can use Pandas and Numpy. Using scikit-learn or any other machine learning library for the questions in this section will give ZERO marks. No excuse will be accepted.**

Regularisation is a technique commonly used in Machine Learning to prevent overfitting. It consists on adding terms to the objective function such that the optimisation procedure avoids solutions that just learn the training data. Popular techniques for regularisation in Supervised Learning include [Lasso Regression](https://en.wikipedia.org/wiki/Lasso_(statistics)), [Ridge Regression](https://en.wikipedia.org/wiki/Tikhonov_regularization) and the [Elastic Net](https://en.wikipedia.org/wiki/Elastic_net_regularization). 

In this part of the Assignment, you will be looking at Ridge Regression and implementing equations to optimise the objective function using the update rules for stochastic gradient descent. You will use those update rules for making predictions on the Air Quality dataset.

## 1.1 Ridge Regression

Let us start with a data set for training $\mathcal{D} = \{\mathbf{y}, \mathbf{X}\}$, where the vector $\mathbf{y}=[y_1, \cdots, y_N]^{\top}$ and $\mathbf{X}$ is the design matrix from Lab 4, this is, 

\begin{align*}
    \mathbf{X} = 
                \begin{bmatrix}
                        1 & x_{1,1} & \cdots & x_{1, D}\\
                        1 & x_{2,1} & \cdots & x_{2, D}\\
                   \vdots &  \vdots\\
                        1 & x_{N,1} & \cdots & x_{N, D}
                \end{bmatrix}
               = 
               \begin{bmatrix}
                      \mathbf{x}_1^{\top}\\
                       \mathbf{x}_2^{\top}\\
                          \vdots\\
                        \mathbf{x}_N^{\top}
                \end{bmatrix}.
\end{align*}

Our predictive model is going to be a linear model

$$ f(\mathbf{x}_i) = \mathbf{w}^{\top}\mathbf{x}_i,$$

where $\mathbf{w} = [w_0\; w_1\; \cdots \; w_D]^{\top}$.

The **objective function** we are going to use has the following form

$$ E(\mathbf{w}, \lambda) = \frac{1}{N}\sum_{n=1}^N (y_n - f(\mathbf{x}_n))^2 + \frac{\lambda}{2}\sum_{j=0}^D w_j^2,$$

where $\lambda>0$ is known as the *regularisation* parameter.

This objective function was studied in Lecture 4. 

The first term on the rhs is what we call the "fitting" term whereas the second term in the expression is the regularisation term. Given $\lambda$, the two terms in the expression have different purposes. The first term is looking for a value of $\mathbf{w}$ that leads the squared-errors to zero. While doing this, $\mathbf{w}$ can take any value and lead to a solution that it is only good for the training data but perhaps not for the test data. The second term is regularising the behavior of the first term by driving the $\mathbf{w}$ towards zero. By doing this, it restricts the possible set of values that $\mathbf{w}$ might take according to the first term. The value that we use for $\lambda$ will allow a compromise between a value of $\mathbf{w}$ that exactly fits the data (first term) or a value of $\mathbf{w}$ that does not grow too much (second term).

This type of regularisation has different names: ridge regression, Tikhonov regularisation or $\ell_2$ norm regularisation. 

## 1.2 Optimising the objective function with respect to $\mathbf{w}$

There are two ways we can optimise the objective function with respect to $\mathbf{w}$. The first one leads to a closed form expression for $\mathbf{w}$ and the second one using an iterative optimisation procedure that updates the value of $\mathbf{w}$ at each iteration by using the gradient of the objective function with respect to $\mathbf{w}$,
$$
\mathbf{w}_{\text{new}} = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}},
$$
where $\eta$ is the *learning rate* parameter and $\frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}}$ is the gradient of the objective function.

It can be shown (this is a question in the Exercise Sheet 4) that a closed-form expression for the optimal $\mathbf{w}_*$ is given as

\begin{align*}            
            \mathbf{w}_*& = \left(\mathbf{X}^{\top}\mathbf{X} + \frac{\lambda N}   
                                     {2}\mathbf{I}\right)^{-1}\mathbf{X}^{\top}\mathbf{y}.
\end{align*}

Alternatively, we can find an update equation for $\mathbf{w}_{\text{new}}$ using gradient descent leading to:

\begin{align*}
   \mathbf{w}_{\text{new}} & = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}
                              {d\mathbf{w}},\\
                           & = \mathbf{w}_{\text{old}} +  \frac{2\eta}{N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n  
                       - \eta\lambda\mathbf{w}_{\text{old}}\\
                           & = (1 - \eta\lambda)\mathbf{w}_{\text{old}} + \frac{2\eta}
                               {N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n
\end{align*}

## 1.3 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 1.a: Missing values and standardisation (2 marks)

* For all the other features with missing values, use the mean value of the non-missing values for that feature to perform imputation. Save these mean values, you will need them when performing the validation stage (**1 mark**).

* Once you have imputed the missing data, we need to standardise the input vectors. Standardise the training data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Keep the mean values and standard deviations, you will need them at validation time (**1 mark**).

#### Question 1.a Answer

In [42]:
data_training_unproc_copy = data_training_unproc
data_training = data_training_unproc
means_dict = {}
for i, column in enumerate(data_training_unproc): 
    # Calculate mean for the column without -200s
    processed_col = data_training_unproc[column].where(data_training_unproc[column] != -200)
    column_mean_wo_200 = np.mean(processed_col)
    # Store means to a hash-table
    means_dict[column] = column_mean_wo_200
    # Replace -200s with the mean previously calculated, assign to a new DataFrame
    data_training[column] = data_training_unproc[column].replace(-200, column_mean_wo_200)

In [43]:
print(means_dict)
data_training.sample(10)

{'CO(GT)': 2.1462025316455704, 'PT08.S1(CO)': 1108.2885489737598, 'C6H6(GT)': 10.224473724456999, 'PT08.S2(NMHC)': 945.1640685892439, 'NOx(GT)': 255.82511820330998, 'PT08.S3(NOx)': 828.5229605092233, 'NO2(GT)': 114.93792627636506, 'PT08.S4(NO2)': 1442.0928650298777, 'PT08.S5(O3)': 1039.8322616263963, 'T': 17.730062999168812, 'RH': 49.008953632968364, 'AH': 0.9880720684021772}


,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
9330,1.4,996.00,5.349138,780.75,177.000000,806.00,123.900000,985.25,630.75,26.450000,16.450000,0.559414
8779,1.5,1257.50,12.054683,1051.50,353.900000,550.50,171.900000,1297.00,1205.00,17.850000,33.275000,0.674092
3288,1.2,988.75,4.036644,712.75,72.000000,954.00,78.000000,1551.50,625.50,27.575000,47.549999,1.726612
1275,2.2,1211.75,9.698220,966.25,152.000000,840.00,100.000000,1690.75,1043.25,19.425001,52.150000,1.164248
4329,0.6,856.25,3.581202,687.00,255.825118,955.25,114.937926,1260.25,725.75,21.900000,48.250000,1.252209
2240,0.6,839.00,4.261422,725.00,36.000000,1052.00,52.000000,1368.25,668.00,24.075000,32.025001,0.946741
1383,3.4,1164.00,14.973260,1147.75,224.000000,741.00,127.000000,1824.25,1129.50,16.050000,50.875000,0.921140
6843,3.7,890.50,7.575617,881.50,342.000000,884.75,149.000000,949.75,893.75,7.750000,33.950000,0.359450
8672,0.9,883.25,1.699793,561.00,92.600000,1079.75,84.400000,785.00,411.75,5.100000,51.175001,0.453449
1992,3.5,1352.75,19.850891,1292.25,184.000000,683.25,95.000000,2177.75,1325.50,23.050000,53.825001,1.496834


In [44]:
# Standardization
processed_means = {}
processed_stds = {}

for i, column in enumerate(data_training):
    # Substract mean
    mean = np.mean(data_training[column])
    processed_means[column] = mean # Store
    data_training[column] -= mean
    # Divide by standard deviation
    std = np.std(data_training[column])
    processed_stds[column] = std # Store
    data_training[column]  /= std

data_training

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
778,-1.279941,-1.662906,-1.318960,-1.865738,-1.145278e+00,3.916685,-1.922859e+00,-1.420554,-1.950331,-0.447102,-1.068806,-1.276605
8361,-1.210613,-1.234079,-1.327433,-1.894685,-5.564809e-15,1.966186,9.722430e-15,-1.556895,-1.667649,-1.769249,2.209604,-0.884531
7899,1.354535,0.806061,0.744035,0.860036,1.904338e+00,-1.015770,2.068759e+00,-0.719891,1.822577,-1.279243,-0.555267,-1.521433
6964,1.631848,2.032949,0.771937,0.884158,1.228131e+00,-1.214961,2.578853e-01,0.290639,1.155984,-0.850125,1.972699,0.133657
146,0.730580,1.706947,0.764109,0.877404,-4.289573e-01,0.101477,5.999629e-01,0.973804,0.998089,0.312553,-0.725956,-0.299683
...,...,...,...,...,...,...,...,...,...,...,...,...
4702,0.522595,0.559515,0.814642,0.920823,2.539351e-01,-0.675567,7.710017e-01,0.822152,0.123941,1.756477,-1.252738,0.952838
3642,-0.378673,-0.357730,-0.265120,-0.129927,-8.778515e-01,-0.256476,-9.393862e-01,0.639148,-0.545199,1.921746,-1.032020,1.748046
8576,-0.725314,-0.570002,-0.927685,-1.015683,-4.800548e-01,0.183323,4.408683e-02,-1.627618,0.419781,-1.611713,-0.184950,-1.566800
8317,-1.141284,-0.881203,-1.261595,-1.693991,-9.322919e-01,1.236474,-1.264360e+00,-1.567832,-1.494475,-1.151667,0.392353,-1.010063


## 1.4 Training and validation stages

We have now curated our training data by removing data observations and features with a large amount of missing values. We have also normalised the feature vectors. We are now in a good position to work on developing the prediction model and validating it. We will use gradient descent for iterative optimisation. 

We first organise the dataframe into the vector of targets $\mathbf{y}$, call it `yTrain`, and the design matrix $\mathbf{X}$, call it `XTrain`.

In [45]:
yTrain = data_training.iloc[:,0].to_numpy().reshape(-1,1)
n_samples, n_features = data_training.shape

# We do not change the shape as the target variable will be replaced by 1s
XTrain = np.ones((n_samples, n_features))
print(XTrain.shape)

for i in range(1, n_features):
    XTrain[:,i] = data_training.iloc[:,i].to_numpy()

(5372, 12)


### Question 1.b: finding the optimal $\mathbf{w}$ with stochastic gradient descent (3 marks)

Use gradient descent to iteratively compute the value of $\mathbf{w}_{\text{new}}$. Instead of using all the training set in `XTrain` and `yTrain` to compute the gradient, use a subset of $S$ instances in `XTrain` and `yTrain`. This is sometimes called *minibatch gradient descent* where $S$ is the size of the minibacth. When using gradient descent with minibatches, you need to find the best values for three parameters: $\eta$, the learning rate, $S$, the number of datapoints in the minibatch and $\gamma$, the regularisation parameter.

* In this question we will use the validation data. So before proceeding to the next steps, make sure that you:  replace the missing values on each feature variables with the mean value you computed with the training data; standardise the validation data using the means and standard deviations computed from the training data (**1 mark**).
    
* Create a grid of values for the parameters $\gamma$ and $\eta$ using `np.logspace` and a grid of values for $S$ using `np.linspace`. Because you need to find three parameters, start with five values for each parameter in the grid and see if you can increase it. Make sure you understand what is the meaning of `np.logspace` and `np.linspace`. Notice that you can use negative values for `start` in `np.logspace` (**1 mark**).

* For each value that you have of $\gamma$, $\eta$ and $S$ from the previous step, use the training set to compute $\mathbf{w}$ using minibatch gradient descent and then measure the RMSE over the validation data. For the validation data, make sure you preprocess it before applying the prediction model over it. For the minibatch gradient descent choose to stop the iterative procedure after $200$ iterations (**1 mark**).

* Choose the values of $\gamma$, $\eta$ and $S$ that lead to the lowest RMSE and save them. You will use them at the test stage.

#### Question 1.b Answer

In [46]:
data_validation = data_val_unproc

In [47]:
# Replace missing values on each feature
for i, column in enumerate(data_val_unproc): 
    mean = means_dict[column]
    data_validation[column] = data_val_unproc[column].replace(-200, mean)

In [48]:
# Standardization
for i, column in enumerate(data_validation):
    # Substract mean
    mean = processed_means[column]
    data_validation[column] -= mean
    # Divide by standard deviation
    std = processed_stds[column]
    data_validation[column] /= std

In [49]:
# Vectorisation
yVal = data_validation.iloc[:,0].to_numpy().reshape(-1,1)
n_samples_val, n_features_val = data_validation.shape

# We do not change the shape as the target variable will be replaced by 1s
XVal = np.ones((n_samples_val, n_features_val))

for i in range(1, n_features_val):
    XVal[:,i] = data_validation.iloc[:,i].to_numpy() 

In [50]:
def RMSE(predictions, targets):
    return np.sqrt(np.mean(np.power(predictions-targets,2)))

In [51]:
import math

NE = 5
learning_rates = np.logspace(-2, -1, NE)
reg_params = np.logspace(-2, -1, NE)
batch_sizes = np.linspace(50, 256, NE)
yVal = yVal.reshape(1,-1)

num_epochs = 200
initializer = True

for i in range(NE): # Iterate over learning rates
    l = learning_rates[i]
    for j in range(NE): # Iterate over Penalty Parameters
        r = reg_params[j]
        for k in range(NE): # Iterate over Batch Size
            batch_size = batch_sizes[k]
            num_batches = int(math.ceil(n_samples/batch_size)) - 1 # Figure out number of batches to iterate over
            w = np.random.rand(n_features, 1)
            for epoch in range(num_epochs): # Iterate over Epochs
                shuffled_samples = np.random.permutation(n_samples)
                for batch in range(num_batches): # Iterate over Batches
                    err = 0
                    for x in range(int(batch_size)): # Iterate over Mini-Batches
                        sample_index = shuffled_samples[int(batch)*int(batch_size) + x]
                        sample = XTrain[sample_index]
                        y_o = yTrain[sample_index]
                        sample = sample.reshape(1,12) # Coz .T or np.transpose does not work as expected on 1ranks
                        y_o = y_o.reshape(1,1)
                        a = y_o - np.dot(sample,w)
                        err += a * sample.T
                    left = (1-(l*r))*w
                    mid = (2*l / batch_size)
                    w_new = left + (mid * err)
                    w = w_new
            # Calculate RMSE
            predics = np.dot(w.T, XVal.T)
            current_rmse = RMSE(predics, yVal)
            if initializer:
                best_rmse = current_rmse
                best_l, best_r, best_k = l, r, batch_size
                initializer = False
                print('RMSE :', best_rmse)
            if current_rmse < best_rmse:
                best_l, best_r, best_k = l, r, batch_size
                best_rmse = current_rmse
                print('RMSE :', best_rmse)
print('Best Combination')
print('Learning Rate :', best_l)
print('Regularisation Param', best_r)
print('Best Batch Size', best_k)

RMSE : 0.3568726513949932
RMSE : 0.35654406579085496
RMSE : 0.35607307606473215
RMSE : 0.3556495152150059
Best Combination
Learning Rate : 0.1
Regularisation Param 0.01
Best Batch Size 153.0


## 1.5 Test stage 

We now know which one is the best model, according to the validation data. We will now put together the training data and the validation data, perform the preprocessing as we did before, this is, treat the missing values and standardise the inputs. We will train the model again using the minibatch stochastic gradient descent and finally compute the RMSE over the test data.


### Question 1.c: combine the original training and original validation data and perform the preprocessing again to this new data (2 marks)

Put together the original training and validation dataset and perform the same preprocessing steps than before, these are: 

* for each feature, impute the missing values with the mean values of the non-missing values (**1 mark**) 

* stardardise the new training set (**1 mark**).

#### Question 1.c Answer

In [52]:
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

print(len(data_training_unproc))
print(len(data_val_unproc))

combined_data = data_training_unproc.append(data_val_unproc)
data_training = combined_data
print(len(data_training))

# Imputation
for i, column in enumerate(combined_data): 
    # Calculate mean for the column without -200s
    processed_col = combined_data[column].where(combined_data[column] != -200)
    column_mean_wo_200 = np.mean(processed_col)
    # Replace -200s with the mean previously calculated, assign to a new DataFrame
    data_training[column] = combined_data[column].replace(-200, column_mean_wo_200)

# Standardization
for i, column in enumerate(data_training):
    # Substract mean
    mean = np.mean(data_training[column])
    data_training[column] -= mean
    # Divide by standard deviation
    std = np.std(data_training[column])
    data_training[column] /= std

data_training

5372
1151
6523


,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
778,-1.281113,-1.672125e+00,-1.321857e+00,-1.868161e+00,-1.145071e+00,3.944866e+00,-1.921461e+00,-1.424121e+00,-1.947084e+00,-0.446457,-1.073546e+00,-1.284350e+00
8361,-1.211810,-1.241665e+00,-1.330340e+00,-1.897118e+00,-6.817000e-15,1.981225e+00,7.303367e-15,-1.560475e+00,-1.665221e+00,-1.767729,2.198488e+00,-8.908319e-01
7899,1.352381,8.062474e-01,7.434064e-01,8.585930e-01,1.918317e+00,-1.020823e+00,2.076477e+00,-7.233976e-01,1.814900e+00,-1.278047,-5.610056e-01,-1.530081e+00
6964,1.629590,2.037810e+00,7.713394e-01,8.827236e-01,1.239056e+00,-1.221357e+00,2.627366e-01,2.872199e-01,1.150237e+00,-0.849213,1.962043e+00,1.311081e-01
146,0.728659,1.710566e+00,7.635027e-01,8.759670e-01,-4.255156e-01,1.039523e-01,6.053558e-01,9.704439e-01,9.927996e-01,0.312695,-7.313628e-01,-3.038285e-01
...,...,...,...,...,...,...,...,...,...,...,...,...
8091,1.629590,-2.986931e-14,6.129114e-15,-1.316797e-14,1.677985e+00,-1.173760e-14,2.359138e+00,2.122138e-14,2.309487e-15,0.000000,2.087001e-15,1.503952e-14
1506,0.797961,1.609695e+00,1.724949e+00,1.640423e+00,-2.288371e-01,-1.000968e+00,4.859962e-02,2.026999e+00,1.476538e+00,0.489444,-3.436532e-01,3.507364e-01
8511,-0.657391,-1.059863e+00,-9.829958e-01,-1.106600e+00,-1.861435e-01,4.553825e-01,1.620922e-01,-1.753702e+00,-1.304639e+00,-1.741651,1.615289e-02,-1.579121e+00
3480,0.035634,-4.294408e-02,4.767757e-01,6.211482e-01,-6.030060e-01,-6.058573e-01,1.247767e+00,1.044089e+00,-4.196902e-01,2.059903,-1.374608e+00,1.209739e+00


### Question 1.d: Preprocess the test data, train the model and predict over the test data (3 marks)

Preprocess the test data and train a new model using the new training set. Finally, report the RMSE over the test set:

* Preprocess the test data by imputing the missing data and standardising it (**1 mark**). 

* Use the best values of $\gamma$, $\eta$ and $S$ found in the validation set and train a new regularised linear model with stochastic gradient descent (**1 mark**).

* Report the RMSE over the test data (**1 mark**).

#### Question 1.d Answer

In [53]:
data_testing = data_test_unproc

# Imputation
for i, column in enumerate(data_test_unproc): 
    # Calculate mean for the column without -200s
    processed_col = data_test_unproc[column].where(data_test_unproc[column] != -200)
    column_mean_wo_200 = np.mean(processed_col)
    # Replace -200s with the mean previously calculated, assign to a new DataFrame
    data_testing[column] = data_test_unproc[column].replace(-200, column_mean_wo_200)

# Standardization
for i, column in enumerate(data_testing):
    # Substract mean
    mean = np.mean(data_testing[column])
    data_testing[column] -= mean
    # Divide by standard deviation
    std = np.std(data_testing[column])
    data_testing[column] /= std

In [54]:
# Vectorisation
yTest = data_validation.iloc[:,0].to_numpy().reshape(-1,1)
n_samples_test, n_features_test = data_testing.shape

# We do not change the shape as the target variable will be replaced by 1s
XTest = np.ones((n_samples_test, n_features_test))

for i in range(1, n_features_test):
    XTest[:,i] = data_testing.iloc[:,i].to_numpy() 
    
yTest = yTest.reshape(1,-1)

# Vectorisation of combined Training sets
yTrain = data_training.iloc[:,0].to_numpy().reshape(-1,1)
n_samples, n_features = data_training.shape

# We do not change the shape as the target variable will be replaced by 1s
XTrain = np.ones((n_samples, n_features))
print(XTrain.shape)

for i in range(1, n_features):
    XTrain[:,i] = data_training.iloc[:,i].to_numpy()

(6523, 12)


In [55]:
l = best_l
r = best_r
batch_size = best_k
num_epochs = 3
w = np.random.rand(n_features, 1)
num_batches = int(math.ceil(n_samples/batch_size)) - 1 # Figure out number of batches to iterate over


for epoch in range(num_epochs): # Iterate over Epochs
    print(epoch)
    shuffled_samples = np.random.permutation(n_samples)
    for x in range(n_samples):
        shuffled_samples = np.random.permutation(n_samples)
        for batch in range(num_batches): # Iterate over Batches
            err = 0
            for x in range(int(batch_size)): # Iterate over Mini-Batches
                sample_index = shuffled_samples[int(batch)*int(batch_size) + x]
                sample = XTrain[sample_index]
                y_o = yTrain[sample_index]
                sample = sample.reshape(1,12) # Coz .T or np.transpose does not work as expected on 1ranks
                y_o = y_o.reshape(1,1)
                a = y_o - np.dot(sample,w)
                err += a * sample.T
            left = (1-(l*r))*w
            mid = (2*l / batch_size)
            w_new = left + (mid * err)
            w = w_new
predics = np.dot(w.T, XTest.T)
rmse = RMSE(predics, yTest)
print('RMSE :', best_rmse)

0
1
2
RMSE : 0.3556495152150059


# 2. Random forests (13 marks)

**USE scikit-learn for the questions on this section.**

In section 1, you used a regularised ridge regression model trained with SGD to create a linear predictive model. In this part of the assignment, you will use **scikit-learn** to train a random forest for regression over the air quality dataset.

## 2.1 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 2.a: Pipeline for missing values and standardisation (3 marks)

* Employ the `SimpleImputer` method in Scikit-learn to impute the missing values in each column using the mean value of the non-missing values, instead (**1 mark**).

* Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the `StandardScaler` method (**1 mark**).

* Create a `Pipeline` that you can use to preprocess the data by filling missing values and then standardising the features (**1 mark**).

#### Question 2.a Answer

In [56]:
data_training_unproc = air_quality.iloc[index[0:N], :].copy()
data_training_unproc.sample(20)

,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
8195,0.3,804.50,0.466223,437.25,28.0,1491.00,27.9,738.25,296.25,6.850000,39.775001,0.396562
447,1.5,966.25,5.451803,785.75,118.0,1111.25,98.0,1257.75,705.25,11.775000,37.599999,0.518424
6004,1.4,844.00,5.369604,781.75,217.0,968.75,112.0,1003.00,795.75,11.650000,37.250000,0.509460
6619,2.3,985.75,8.794601,931.25,362.0,814.75,120.0,1011.00,975.50,18.250000,19.475000,0.404424
705,1.0,-200.00,-200.000000,-200.00,-200.0,-200.00,-200.0,-200.00,-200.00,-200.000000,-200.000000,-200.000000
2469,1.6,907.00,9.560280,961.00,105.0,919.75,102.0,1563.75,668.25,31.100000,18.175000,0.807943
7620,0.4,1288.25,11.617169,1036.25,581.8,590.00,183.2,1241.50,1412.50,5.225000,81.424999,0.727627
157,1.6,1172.75,7.486889,877.75,160.0,1038.00,84.0,1556.25,1051.75,9.900000,65.200000,0.795646
6862,2.3,958.75,8.599625,923.50,316.0,899.00,137.0,993.25,897.25,8.775000,35.675000,0.404280
7968,0.9,925.75,4.261422,725.00,195.1,933.25,141.7,847.25,694.25,7.200000,34.950001,0.356711


In [57]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=-200, strategy='mean')
data_training_processed = imp_mean.fit_transform(data_training_unproc)
data_training_processed

array([[3.00000000e-01, 7.52500000e+02, 6.57346480e-01, ...,
        1.38750000e+01, 3.08500004e+01, 4.86762274e-01],
       [4.00000000e-01, 8.44250000e+02, 5.95883757e-01, ...,
        2.47499996e+00, 8.65500011e+01, 6.40725584e-01],
       [4.10000000e+00, 1.28075000e+03, 1.56213592e+01, ...,
        6.70000005e+00, 3.95749998e+01, 3.90620751e-01],
       ...,
       [1.10000000e+00, 9.86333333e+02, 3.49547239e+00, ...,
        3.83333325e+00, 4.58666674e+01, 3.72805890e-01],
       [5.00000000e-01, 9.19750000e+02, 1.07344309e+00, ...,
        7.80000019e+00, 5.56750002e+01, 5.91430621e-01],
       [7.00000000e-01, 8.80750000e+02, 2.25031870e+00, ...,
        1.17999997e+01, 8.13750000e+01, 1.12380283e+00]])

In [58]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # According to documentation, the default is exactly what we want
scaler.fit_transform(data_training_processed)

array([[-1.27994089, -1.66290603, -1.31895966, ..., -0.44710189,
        -1.06880607, -1.27660513],
       [-1.21061259, -1.23407934, -1.32743314, ..., -1.76924929,
         2.20960421, -0.8845315 ],
       [ 1.35453464,  0.80606076,  0.74403466, ..., -1.2792429 ,
        -0.55526698, -1.5214333 ],
       ...,
       [-0.72531447, -0.57000166, -0.92768509, ..., -1.61171273,
        -0.18494979, -1.56679954],
       [-1.14128429, -0.88120287, -1.26159503, ..., -1.15166726,
         0.39235252, -1.01006306],
       [-1.00262768, -1.06348343, -1.0993466 , ..., -0.68775594,
         1.90501217,  0.34564373]])

In [59]:
from sklearn.pipeline import Pipeline
my_pipeline = Pipeline([('imputer', SimpleImputer(missing_values=-200, strategy='mean'))
                        , ('scaler', StandardScaler())])

### Question 2.b: Use the Pipeline to fit the training data and transform the validation data (2 marks)

In the previous question, you created a `Pipeline` for applying a `SimpleImputer` and a `StandardScaler`. Use the Pipeline to fit the training data (**1 mark**) and transform the validation data (**1 mark**).

#### Question 2.b Answer

In [60]:
data_training_unproc = air_quality.iloc[index[0:N], :].copy()
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy()

data_training_processed = my_pipeline.fit_transform(data_training_unproc)
data_validation_processed = my_pipeline.transform(data_val_unproc)

print(data_validation_processed)

[[ 0.17595348  0.35386476 -0.03048995 ...  0.2197707   0.02595925
   0.32982271]
 [ 0.93856482  1.66605108  1.08060324 ...  0.01680948 -0.40223524
  -0.33705011]
 [-0.44800125  0.32932699 -0.7241829  ... -0.83852713  0.52478381
  -0.64404855]
 ...
 [-0.65598616 -1.05296724 -0.98047051 ... -1.74315428  0.02301632
  -1.57029344]
 [ 0.03729687 -0.03990799  0.47769695 ...  2.06091895 -1.37045516
   1.20832742]
 [-0.79464277 -0.15792296 -0.87661459 ... -0.67615814  1.08540954
  -0.12868654]]


In [61]:
# Vectorisation
yTrain = data_training_processed[:,0].reshape(-1)
yVal = data_validation_processed[:,0].reshape(-1)
XTrain = np.delete(data_training_processed, 0, 1)
XVal = np.delete(data_validation_processed, 0, 1)
print(XTrain.shape)
print(yTrain.shape)
print(XVal.shape)
print(yVal.shape)

(5372, 11)
(5372,)
(1151, 11)
(1151,)


## 2.2 Random forest to predict air quality 

We now use random forests to predict air quality. Remember that the tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. Some of the parameters required in the Random Forest implementation in Scikit-learn include:

Some of the additional parameters required in the Random Forest implementation in Scikit-learn include

> **n_estimators** the total number of trees to train<p>
**max_features** number of features to use as candidates for splitting at each tree node. <p>
    **boostrap**: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.<p>
   **max_samples**: If bootstrap is True, the number of samples to draw from X to train each base estimator.

### Question 2.c: train a random forest (4 marks)

In this question, you will train a random forest for predicting over the validation data. Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor. Parameters to include in your exploration are **n_estimators**, **max_features** and **max_samples**. Use `np.linspace` or `np.logspace` to define ranges of values to explore for each parameter and create a grid to be assessed over the validation data. Make sure you use the same validation data that was given to you.

* Use `PredefinedSplit` to tell the cross validator which instances to use for training and which ones for validation (**1 mark**).

* Create a grid of values to explore that include a range of at least five values for each parameter **n_estimators**, **max_features** and **max_samples** (**1 mark**). 

* Train a random forest for regression model that uses the grid of parameters you created before. Use `GridSearchCV` to find the best set of parameters by performing cross-validation over the predefined split. (**1 mark**).

* Print the best values in your grid for **n_estimators**, **max_features** and **max_samples** (**1 mark**).

#### Question 2.c Answer

In [62]:
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

test_fold = []

for x in range(len(data_training_processed)):
    test_fold.append(-1)
    
for y in range(len(data_validation_processed)):
    test_fold.append(0)
    
ps = PredefinedSplit(test_fold)

N_params = 5
n_estimators = np.linspace(100, 200, N_params)
max_features = np.linspace(5, 10, N_params)
max_samples = np.logspace(-1, -.01, N_params)
print(max_samples)

parameters = {'n_estimators': n_estimators.astype(int), 'max_features': max_features.astype(int), 
              'max_samples': max_samples}

model = RandomForestRegressor()
clf = GridSearchCV(model, parameters)
print(clf)
clf.fit(XTrain, yTrain)

[0.1        0.17680722 0.31260794 0.55271341 0.97723722]
GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_features': array([ 5,  6,  7,  8, 10]),
                         'max_samples': array([0.1       , 0.17680722, 0.31260794, 0.55271341, 0.97723722]),
                         'n_estimators': array([100, 125, 150, 175, 200])})


GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_features': array([ 5,  6,  7,  8, 10]),
                         'max_samples': array([0.1       , 0.17680722, 0.31260794, 0.55271341, 0.97723722]),
                         'n_estimators': array([100, 125, 150, 175, 200])})

In [63]:
print(clf.best_params_)

{'max_features': 5, 'max_samples': 0.5527134079444348, 'n_estimators': 175}


In [64]:
predics = clf.predict(XVal)
rmse = RMSE(predics, yVal)
print(rmse)

0.2905272955892348


### Question 2.d: train a new model over the whole training data and report the prediction over the test set (4 marks)


Now that we have identified the best paramaters of the regression model, we use these parameters to train a new model over the whole training data (`data_training` plus `data_val`). We apply this model to the test set and report the performance.

* Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data (**1 mark**).

* Apply the created preprocessing pipeline to the test data (**1 mark**).

* Fit a random forest regression model to the training data using the best parameters found in Question 2.c (**1 mark**).

* Compute the RMSE over the test data and report the result (**1 mark**).

#### Question 2.d Answer

In [65]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

data_training_unproc = air_quality.iloc[index[0:N], :].copy()
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy()

my_pipeline = Pipeline([('imputer', SimpleImputer(missing_values=-200, strategy='mean'))
                        , ('scaler', StandardScaler())])

whole_training_data = pd.concat([data_training_unproc, data_val_unproc])
whole_processed_training_data = my_pipeline.fit_transform(whole_training_data)

# Vectorisation
yTrain = whole_processed_training_data[:,0].reshape(-1)
XTrain = np.delete(whole_processed_training_data, 0, 1)
print(XTrain.shape)
print(yTrain.shape)

(6523, 11)
(6523,)


In [66]:
# Test Data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data
data_set_proc = my_pipeline.fit_transform(data_test_unproc)

# Vectorisation
yTest = data_set_proc[:,0].reshape(-1)
XTest = np.delete(data_set_proc, 0, 1)

In [67]:
my_model = RandomForestRegressor()
params = clf.best_params_
print(params)
my_model.set_params(**params)
my_model.fit(XTrain, yTrain)

predics = clf.predict(XTest)
rmse = RMSE(predics, yTest)
print(rmse)

{'max_features': 5, 'max_samples': 0.5527134079444348, 'n_estimators': 175}
0.28139034901255855
